In [39]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
from pandas.api.types import union_categoricals
from itertools import islice
import re
import addfips
import requests
import urllib
import time
DC_STATEHOOD = 1 # Enables DC to be included in the state list
import us
import pickle
import rapidfuzz

Use FRED API to get state-level non-farm minimum wage

In [40]:
# FRED API key from my account
# Import API key stored in text file
with open("../tools/fred_api_key.txt") as f:
    lines = f.readlines()

api_key = lines[0]

In [41]:
# Get release ID of state minimum wages
# Grab release ID of all releases
fred_release_api_url = 'https://api.stlouisfed.org/fred/releases'
release_params = {
    'api_key': api_key,
    'file_type': 'json'
    }

fred_release = requests.get(fred_release_api_url, params = release_params)
release_list = fred_release.json()['releases']

# Take the release id of the state minimum wage series
state_minimum_wage_release = list(filter(lambda release_dict: release_dict['name'] == 'Minimum Wage Rate by State', release_list))
state_minimum_wage_release_id = state_minimum_wage_release[0]['id']
state_minimum_wage_release_id

387

In [42]:
# Base URL to call API
fred_observations_api_url = 'https://api.stlouisfed.org/fred/v2/release/observations'

# Bearer token has to go in request header
fred_observations_headers = {
    'Authorization': f'Bearer {api_key}'
}

fred_observations_params = {
    'release_id': '387',
    'format': 'json'
}

In [43]:
# Get all state-level observations
response = requests.get(fred_observations_api_url, headers= fred_observations_headers, params = fred_observations_params)
list_obs = response.json()['series']

In [44]:
# Convert from JSON to dataframe
df = pd.json_normalize(list_obs).reset_index(drop=True)
df = df.explode('observations', ignore_index = True)

In [45]:
# Observations column is still in JSON, unpack that as well as a separate dataframe, then merge back into original dataframe
obs = pd.json_normalize(df.pop('observations')).reset_index(drop=True)
df = pd.concat([df, obs], axis = 1)

Preserve only information we want in dataframe

In [46]:
# Add state name, abbreviation, and year
df['state_name'] = df['title'].str.split('for ', expand = True)[1]
df['year'] = df['date'].str[0:4]
df['state_abbreviation'] = df['series_id'].str[-2:]

# Federal minimum wage
fed = df['series_id'] == 'STTMINWGFG'
df.loc[fed, 'state_name'] = 'USA'
df.loc[fed, 'state_abbreviation'] = 'US'

# Georgia has monthly observations that we can remove
df = df[df['frequency'] == 'Annual']

In [47]:
min_wage_df = df[['state_name', 'state_abbreviation', 'year', 'value', 'units', 'seasonal_adjustment']].copy()
# min_wage_df = min_wage_df[min_wage_df['year'] > '1999']

In [48]:
df.to_csv('min_wage.csv')

Add agricultural worker exemption data

In [49]:
ag_exemption_list = ['AK', 'DE', 'GA', 'IN', 'KS', 'KY', 'ME', 'MA', 'NE', 'NH', 'NJ', 'OK', 'RI', 'VT', 'VA', 'WA', 'WV', 'WY']
min_wage_df['agriculture_exemption'] = min_wage_df['state_abbreviation'].isin(ag_exemption_list)

In [50]:
# Add state FIPS code and export
min_wage_df['state_fips'] = min_wage_df['state_name'].map(us.states.mapping('name', 'fips'))
min_wage_df = min_wage_df.rename(columns={'state_fips':'state_fips_code'})

In [51]:
output_file = 'state_year_min_wage.parquet'
output_dir = Path('../binaries')
output_dir.mkdir(parents=True, exist_ok=True)
min_wage_df.to_parquet(output_dir / output_file, index=False)  # can join path elements with / operator

output_dir = Path('../files_for_phil')
output_dir.mkdir(parents=True, exist_ok=True)
min_wage_df.to_parquet(output_dir / output_file, index=False)

In [52]:
min_wage_df

,state_name,state_abbreviation,year,value,units,seasonal_adjustment,agriculture_exemption,state_fips_code
0,Alaska,AK,1968,2.1,Dollars per Hour,Not Seasonally Adjusted,True,02
1,Alaska,AK,1969,2.1,Dollars per Hour,Not Seasonally Adjusted,True,02
2,Alaska,AK,1970,2.1,Dollars per Hour,Not Seasonally Adjusted,True,02
3,Alaska,AK,1971,2.1,Dollars per Hour,Not Seasonally Adjusted,True,02
4,Alaska,AK,1972,2.1,Dollars per Hour,Not Seasonally Adjusted,True,02
...,...,...,...,...,...,...,...,...
3283,Wyoming,WY,2019,7.25,Dollars per Hour,Not Seasonally Adjusted,True,56
3284,Wyoming,WY,2020,7.25,Dollars per Hour,Not Seasonally Adjusted,True,56
3285,Wyoming,WY,2021,7.25,Dollars per Hour,Not Seasonally Adjusted,True,56
3286,Wyoming,WY,2022,7.25,Dollars per Hour,Not Seasonally Adjusted,True,56
